## Jak predobrzyłem w rywalizacji przeciwko dziennikarzom sportowym?

W czewrcu 2024 roku dostałem zaproszenie do firmowej ligi fantasy przewidywania meczów Euro 2024. A że moim pracodawcą był Eurosport, to moimi rywalami byli dziennikarze sportowi. Czułem, że przeciwko nim moje szanse są marne, więc postanowiłem wsprzeć się Pythonem.

Liga fantasy polegała na przewidywaniu dokładnych wyników meczów. Punkty zdobywało się przy poprawnym obstawieniu wyniku, zwycięzcy, liczby strzelonych goli etc.

Od razu pomyślałem o wykorzystaniu do tego rankigu elo. Wiedziałem, że są prowadzone rankingi drużyn narodowych wg. elo [biorące pod uwagę całą historę meczów międzynarodowych]. (https://eloratings.net)

Musiałem jednak zrosumieć dokładnie, jak można w ten sposób obliczyć szane na zwycięstwo jednej czy drugiej strony. 

Poniższy wzór podaje oczekiwany wynik gracza A (w skali 1 za zwycięstwo, 0 za porażkę), gdzie R_a - to liczba punktów Elo gracza A i R_b - to liczba punktów Elo gracza B.

![](img.svg)

In [1]:
#Rannking Holandii przed Euro
elo1 = 1920
#Ranking Polski przed Euro
elo2 = 1770

p_1 = 1/(1 + 10**((elo2-elo1)/400))
p_1

0.7033850034718286

Czy to oznacza, że Holandia miała 70% szans na zwycięstwo z Polską? Nie. To był jej oczekiwany wynik, na który składa się szansa na zwycięstwo oraz 1/2 szansy na remis.

Jak w takim razie obliczyć szansę na remis? W originalnym systemie Elo nie ma na to żadnego wzoru, natomiast w jednym z artykułów naukowych znalazłam taki wzór. 

![](imgelo.jpg)

Gdzie dr to różnica pomiędy elo obu graczy.

Xiong, Shuo & Yang, Liang & Azan, Nor & Mat Zin, Nor Azan & Iida, Hiroyuki. (2016). Mathematical Model of Ranking Accuracy and Popularity Promotion. 10.1109/ICSAI.2016.7810980. 
[dostęp: 26.09.2024] https://www.researchgate.net/publication/309662241_Mathematical_Model_of_Ranking_Accuracy_and_Popularity_Promotion
Początkowo znaleziono tutaj: https://www.reddit.com/r/excel/comments/kivm21/calculating_probability_for_a_draw_based_on_elo/

In [2]:
import numpy as np

In [3]:
p_1 = 1/(1 + 10**((elo2-elo1)/400))
p_2 = 1 - p_1

dr = elo1-elo2
dr = 200
p_d = (1/(np.sqrt(2 * np.pi*np.e))) * (np.exp((-((dr/200)**2) / (2*(np.e**2)))))

print(f"Szansa na remis: {p_d}")


Szansa na remis: 0.2261388303282616


In [4]:
import numpy as np

def P_draw_elo(dr, e):
    # Constants
    pi = np.pi
    factor = 200
    
    # Compute the probability
    result = (1 / np.sqrt(2 * pi * e**2)) * np.exp(-(dr / factor)**2 / (2 * e**2))
    return result

# Example usage with sample values
dr = 289  # Example value for dr
e = np.e  # Example value for e

# Calculate the probability
probability = P_draw_elo(dr, e)
print(f"P(draw elo) = {probability}")


P(draw elo) = 0.12742462564645288


Znając to możemy obliczyć szanse na zwycięstwo obu krajów.

In [6]:
p_1w = p_1 - 1/2*p_d
p_2w = p_2 - 1/2*p_d

print(f"Szansa na zwycięstwo Holandii: {p_1w*100:.2f}%. Szansa na zwycięstwo Polski: {p_2w*100:.2f}%")

Szansa na zwycięstwo Holandii: 59.03%. Szansa na zwycięstwo Polski: 18.35%


Obliczyliśmy prawdopodobieństwo zwycięstwa. Natomiast przed nami kolejny problem. Typer wymagał od nas podania dokładnego wyniku spotkania. Jak to zrobić? Istnieje kilka podejśc, logicznie im większa róznica elo pomiędzy drużynami, tym większa szana na większą różnice goli. Natomiast, przyjąłem "naiwne" założenie, że na Euro, jako dużym turnieju, bardziej zależy drużynom żęby nie przegrać za wszelką cenę, i to zniweluje potencjalną większą ilośc bramek w meczu związaną z różnicą klas zespołów.

Zdeccydowałem się więc losować z dystrybucji wyników historycznych meczów. Nie brałem pod uwagę przewagi grania meczów u siebie, bądź na wyjeździe, bo Euro jest rozgrywane na neutralnym gruncie. Tak więc dla meczów, w któ®ych jedna z drużyn wygrywa losuje z dystrybucji meczó, w których wygrała drużyna gospodarzy.

In [14]:
import random

Na przykład wynik 1-1 pada 11% meczów, a drużyna gospodarzy wygrywa 4-0 w 2% spotkań. 

In [15]:
remis = {"1-1": 0.11, "0-0": 0.077, "2-2": 0.047, "3-3": 0.01, "4-4": 0.001}
home = {"1-0": 0.105, "2-1": 0.085, "2-0": 0.073,"3-0": 4.3/100, "3-1": 4.2/100, "3-2": 2.5/100, "4-0": 0.02, "4-1": 0.018, "4-2" : 0.01, "5-0": 0.008, "4-3": 0.004}

za https://footystats.org/stats/common-score

Przykładowo, jeśli kod uzna, że będzie remis, to wynik będzie losowany w poniższy sposób. 

In [26]:
# Losuję wartość z przedziału (0, suma prawdopodobieństw remisów)
# wartość pomiędzy (0,0.11) oznacza 1-1, (0.11, 0.187) 0-0 itd.
suma = sum(remis.values())
test = random.uniform(0,suma)
for ele in remis:
    if remis[ele] >= test:
        wynik = ele
        break
    else:
        test -= remis[ele]
print(wynik)

0-0


In [28]:
def wygeneruj_wynik(elo1, elo2):
    # Liczę oczekiwany wynik każdego gracza: P zwycięstwa + 1/2 * P remisu
    p_1 = 1/(1 + 10**((elo2-elo1)/400))
    p_2 = 1 - p_1
    
    dr = elo1-elo2
    #dr = 289
    gora_exp = (dr/200)**2
    dol_exp = (2*np.e**2)
    exxxp = (-(gora_exp/dol_exp))
    stala = 1/(np.sqrt(2 * np.pi*np.e))
    # P remisu
    p_d = stala * (np.exp(exxxp))
    
    p_1w = p_1 - 1/2*p_d
    p_2w = p_2 - 1/2*p_d
    #print(p_d)

    # Losowanie zwycięstwy za pomocą obliczonych prawdopodobieństw
    zwyciezca = random.choices([1,0,2], weights = [p_1w, p_d, p_2w], k=1)[0]

    # Jeśli remis
    if zwyciezca == 0:
        suma = sum(remis.values())
        test = random.uniform(0,suma)
        for ele in remis:
            tenruch = remis[ele]
            if remis[ele] >= test:
                wynik = ele
                break
            else:
                test -= remis[ele]
    # Jeśli mamy zwycięzcę
    else:
        suma = sum(home.values())
        test = random.uniform(0,suma)
        for ele in home:
            tenruch = home[ele]
            if home[ele] >= test:
                wynik = ele
                break
            else:
                test -= home[ele]  

    if zwyciezca == 2:
        wynik = wynik[::-1]
    
    return f"Wynik: {zwyciezca}, rezultat {wynik}"

In [32]:
# Przykład
# Drużyna 1: 1920 elo
# Drużyna 2: 1832 elo
wygeneruj_wynik(1920, 1832)

'Wynik: 1, rezultat 2-1'

In [37]:
# Przewiduję każdą kombinację wyników pomiędzy zespołami 
last4 = {"Hiszpania": 2112, "Francja": 2069, "Holandia": 1989, "Anglia" : 1971}
#grupaA = {"Czechy": 1777, "Turcja": 1749, "Gruzja": 1666, "portugalia" : 2003}

for i in range(3):
    lista = [1,2,3]
    gracze = list(last4.keys())
    print(f"{gracze[0]}, {gracze[lista[i]]}, {wygeneruj_wynik(last4[gracze[0]], last4[gracze[lista[i]]])}")
    del lista[i]
    print(f"{gracze[lista[0]]} - {gracze[lista[1]]}, {wygeneruj_wynik(last4[gracze[lista[0]]], last4[gracze[lista[1]]])}")

Hiszpania, Francja, Wynik: 0, rezultat 1-1
Holandia - Anglia, Wynik: 2, rezultat 2-3
Hiszpania, Holandia, Wynik: 1, rezultat 3-0
Francja - Anglia, Wynik: 1, rezultat 2-1
Hiszpania, Anglia, Wynik: 0, rezultat 2-2
Francja - Holandia, Wynik: 0, rezultat 0-0
